In [1]:
import logging

logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

In [2]:
import mglearn

from sklearn.model_selection import train_test_split 

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
# import matplotlib.font_manager as fm
import seaborn as sns

# 브라우저에서 바로 그려지도록 
%matplotlib inline

# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

# Colab 의 한글 폰트 설정
plt.rc('font', family='NanumBarunGothic') 

# 유니코드에서  음수 부호설정
mpl.rc('axes', unicode_minus=False)

# 차원의 저주
> 데이터 학습을 위해 차원이 증가하면서 학습데이터의 수가 차원의 수보다 적어져서 성능이 저하되는 현상

>> 차원을 축소하거나 데이터를 더 많이 수집해야함

# 데이터 로드

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier

SEED = 42

In [12]:
%cd C:\\python

C:\python


In [13]:
DATA_PATH = "datas/"
df = pd.read_csv(DATA_PATH+"Titanic.csv")

df.columns = [col.lower() for col in df.columns] # 컬럼명 소문자로 변환
df.columns

Index(['passengerid', 'survived', 'pclass', 'name', 'sex', 'age', 'sibsp',
       'parch', 'ticket', 'fare', 'cabin', 'embarked'],
      dtype='object')

In [ ]:
y_train = df['survived']  #정답값
df = df.drop('survived',axis=1) #정답값 제외

df.shape, y_train.shape

In [18]:
#결측치 미리 채우기
df.age = df.age.fillna(df.age.median())
df.fare = df.fare.fillna(df.fare.median())
df.cabin = df.cabin.fillna('UNK')  #cabin 임의의 문자열로 채우기
df.embarked = df.embarked.fillna(df.embarked.mode()[0])

In [19]:
#학습에 바로 사용 가능한 특성
cols = ['pclass','age','sibsp','parch','fare']
features =df[cols]

In [22]:
print(f'before"{df.shape}')

#범주형 one-hot-encoding
cols = ['sex','embarked','ticket','cabin']
enc = OneHotEncoder()
sparse_features = pd.DataFrame(
    enc.fit_transform(df[cols]).toarray(),
    columns  = enc.get_feature_names_out()
)
x_train = pd.concat([features, sparse_features],axis=1) #특성
print(f'after: {x_train.shape}')
x_train.head()

before"(891, 11)
after: (891, 839)


,pclass,age,sibsp,parch,fare,sex_female,sex_male,embarked_C,embarked_Q,embarked_S,...,cabin_F E69,cabin_F G63,cabin_F G73,cabin_F2,cabin_F33,cabin_F38,cabin_F4,cabin_G6,cabin_T,cabin_UNK
0,3,22.0,1,0,7.2500,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,38.0,1,0,71.2833,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,26.0,0,0,7.9250,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1,35.0,1,0,53.1000,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3,35.0,0,0,8.0500,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Base model

In [25]:
model = LGBMClassifier(random_state=SEED)
cv =KFold(n_splits =5, shuffle=True, random_state=SEED) #5번 학습 시킨 모델

scores = cross_val_score(model,x_train, y_train,cv=cv, scoring='roc_auc',n_jobs=-1)
base_score = scores.mean()
base_score

0.8766894920348889

## 주성분 분석(PCA)
> 